# ML Flow Dagshub Tutorial

In [2]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [3]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [4]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

### Track Experiments

In [5]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
]

In [6]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [7]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [9]:
import dagshub
dagshub.init(repo_owner='victosdur77', repo_name='my-first-repo', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=a60aeed6-23af-48c6-a288-4a5f5e365ca8&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=5669e73f7e24b328ac9706a601de96b9952b06f40be8dc985707da89a08cb8ad




Output()

Accessing as victosdur77

Initialized MLflow to track repo "victosdur77/my-first-repo"

Repository victosdur77/my-first-repo initialized!

In [10]:
# # Ideally you will not require following 4 lines if you have started fresh and do not have any previous dagshub credentials on your computer
# import os
# os.environ['MLFLOW_TRACKING_USERNAME'] = victosdu' # 'learnpythonlanguage'
# os.environ['MLFLOW_TRACKING_PASSWORD'] = 'your password' # 
# os.environ['MLFLOW_TRACKING_URI'] = 'your dagshub unique uri' # https://dagshub.com/learnpythonlanguage/mlflow_dagshub_demo.mlflow

# Initialize MLflow
mlflow.set_experiment("MlFlow-Dagshub quickstart")
# mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, model_name)
        else:
            mlflow.sklearn.log_model(model, model_name)  

2024/10/31 11:48:05 INFO mlflow.tracking.fluent: Experiment with name 'MlFlow-Dagshub quickstart' does not exist. Creating a new experiment.
2024/10/31 11:48:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/31 11:48:39 INFO mlflow.tracking._tracking_service.client: 🏃 View run Logistic Regression at: https://dagshub.com/victosdur77/my-first-repo.mlflow/#/experiments/0/runs/010fd41471bf44a78cb5320aa49a1f8d.
2024/10/31 11:48:39 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/victosdur77/my-first-repo.mlflow/#/experiments/0.
2024/10/31 11:48:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/10/31 11:48:53 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest at: ht

Now you can find everything as in the mlflow ui in the following link for example: https://dagshub.com/victosdur77/my-first-repo/experiments